# Assignment: Video Quality Inference

To this point in the class, you have learned various techniques for leading and analyzing packet captures of various types, generating features from those packet captures, and training and evaluating models using those features.

In this assignment, you will put all of this together, using a network traffic trace to train a model to automatically infer video quality of experience from a labeled traffic trace.

## Part 1: Warmup

The first part of this assignment builds directly on the hands-on activities but extends them slightly.

### Extract Features from the Network Traffic

Load the `netflix.pcap` file, which is a packet trace that includes network traffic. 

Click [here](https://github.com/noise-lab/ml-systems/blob/main/docs/notebooks/data/netflix.pcap) to download `netflix.pcap`.


In [42]:
from scapy.all import rdpcap, IP, TCP, UDP, DNS
import pandas as pd

pkts = rdpcap("netflix.pcap")  # may use lots of RAM for big files
rows = []
for p in pkts:
    is_dns = DNS in p
    proto = "DNS" if is_dns else ("TCP" if TCP in p else ("UDP" if UDP in p else p.name))
    txid = p[DNS].id if is_dns else None
    rows.append({
        "timestamp": float(getattr(p, "time", None)),
        "length": len(p),
        "src_ip": p[IP].src if IP in p else None,
        "dst_ip": p[IP].dst if IP in p else None,
        "protocol": proto,
        "src_port": p[TCP].sport if TCP in p else (p[UDP].sport if UDP in p else None),
        "dst_port": p[TCP].dport if TCP in p else (p[UDP].dport if UDP in p else None),
        "txid": txid,
        "info": str(p.summary())
    })

pcap_import = pd.DataFrame(rows)

In [43]:
pcap_import.head()

,timestamp,length,src_ip,dst_ip,protocol,src_port,dst_port,txid,info
0,1.518358e+09,77,192.168.43.72,128.93.77.234,DNS,55697.0,53.0,60684.0,"Ether / IP / UDP / DNS Qry ""b'fonts.gstatic.co..."
1,1.518358e+09,77,192.168.43.72,128.93.77.234,DNS,59884.0,53.0,12314.0,"Ether / IP / UDP / DNS Qry ""b'fonts.gstatic.co..."
2,1.518358e+09,87,192.168.43.72,128.93.77.234,DNS,61223.0,53.0,4563.0,"Ether / IP / UDP / DNS Qry ""b'googleads.g.doub..."
3,1.518358e+09,87,192.168.43.72,128.93.77.234,DNS,58785.0,53.0,4740.0,"Ether / IP / UDP / DNS Qry ""b'googleads.g.doub..."
4,1.518358e+09,78,192.168.43.72,128.93.77.234,DNS,51938.0,53.0,13362.0,"Ether / IP / UDP / DNS Qry ""b'ytimg.l.google.c..."


### Identifying the Service Type

Use the DNS traffic to filter the packet trace for Netflix traffic.

In [60]:
nflx_queries = pcap_import[pcap_import["info"].str.contains(r"netflix|nflx", na=False) &
                           pcap_import["info"].str.contains(r"Qry", na=False)]

nflx_txids = [int(x) for x in nflx_queries["txid"].unique().tolist()]

In [68]:
print(nflx_txids)

[45466, 12361, 24416, 7939, 30582, 14313, 37909, 26231, 48727, 21826, 9887, 33001, 49332, 14721, 33500, 34542, 60735, 43441]


In [74]:
nflx_answers = pcap_import[pcap_import["txid"].isin(nflx_txids) & 
                           pcap_import["protocol"].eq("DNS") & 
                           pcap_import["info"].str.contains(r"Ans", na=False)]

nflx_answers.head(10)

,timestamp,length,src_ip,dst_ip,protocol,src_port,dst_port,txid,info
101,1.518358e+09,109,128.93.77.234,192.168.43.72,DNS,53.0,48058.0,30582.0,"Ether / IP / UDP / DNS Ans ""198.38.120.130"""
102,1.518358e+09,102,128.93.77.234,192.168.43.72,DNS,53.0,43209.0,45466.0,"Ether / IP / UDP / DNS Ans ""52.19.39.146"""
103,1.518358e+09,113,128.93.77.234,192.168.43.72,DNS,53.0,4046.0,7939.0,"Ether / IP / UDP / DNS Ans ""52.210.19.176"""
104,1.518358e+09,109,128.93.77.234,192.168.43.72,DNS,53.0,50901.0,14313.0,"Ether / IP / UDP / DNS Ans ""198.38.120.153"""
105,1.518358e+09,96,128.93.77.234,192.168.43.72,DNS,53.0,28162.0,12361.0,"Ether / IP / UDP / DNS Ans ""23.57.80.120"""
108,1.518358e+09,96,128.93.77.234,192.168.43.72,DNS,53.0,48245.0,24416.0,"Ether / IP / UDP / DNS Ans ""23.57.80.120"""
219,1.518358e+09,94,128.93.77.234,192.168.43.72,DNS,53.0,57216.0,26231.0,"Ether / IP / UDP / DNS Ans ""198.38.120.137"""
224,1.518358e+09,96,128.93.77.234,192.168.43.72,DNS,53.0,55348.0,48727.0,"Ether / IP / UDP / DNS Ans ""23.57.80.120"""
228,1.518358e+09,109,128.93.77.234,192.168.43.72,DNS,53.0,15562.0,37909.0,"Ether / IP / UDP / DNS Ans ""198.38.120.167"""
1019,1.518358e+09,108,128.93.77.234,192.168.43.72,DNS,53.0,36897.0,21826.0,"Ether / IP / UDP / DNS Ans ""34.252.77.54"""


### Generate Statistics

Generate statistics and features for the Netflix traffic flows. Use the `netml` library or any other technique that you choose to generate a set of features that you think would be good features for your model. 

**Write a brief justification for the features that you have chosen.**

### Inferring Segment downloads

In addition to the features that you could generate using the `netml` library or similar, add to your feature vector a "segment downloads rate" feature, which indicates the number of video segments downloaded for a given time window.

Note: If you are using the `netml` library, generating features with `SAMP` style options may be useful, as this option gives you time windows, and you can then simply add the segment download rate to that existing dataframe.

## Part 2: Video Quality Inference

You will now load the complete video dataset from a previous study to train and test models based on these features to automatically infer the quality of a streaming video flow.

For this part of the assignment, you will need two pickle files, which we provide for you by running the code below:

```

!gdown 'https://drive.google.com/uc?id=1N-Cf4dJ3fpak_AWgO05Fopq_XPYLVqdS' -O netflix_session.pkl
!gdown 'https://drive.google.com/uc?id=1PHvEID7My6VZXZveCpQYy3lMo9RvMNTI' -O video_dataset.pkl

```

### Load the File

Load the video dataset pickle file.

### Clean the File

1. The dataset contains video resolutions that are not valid. Remove entries in the dataset that do not contain a valid video resolution. Valid resolutions are 240, 360, 480, 720, 1080.

2. The file also contains columns that are unnecessary (in fact, unhelpful!) for performing predictions. Identify those columns, and remove them.

**Briefly explain why you removed those columns.**

### Prepare Your Data

Prepare your data matrix, determine your features and labels, and perform a train-test split on your data.

### Train and Tune Your Model

1. Select a model of your choice.
2. Train the model using your training data.

### Tune Your Model

Perform hyperparameter tuning to find optimal parameters for your model.

### Evaluate Your Model

Evaluate your model accuracy according to the following metrics:

1. Accuracy
2. F1 Score
3. Confusion Matrix
4. ROC/AUC

## Part 3: Predict the Ongoing Resolution of a Real Netflix Session

Now that you have your model, it's time to put it in practice!

Use a preprocessed Netflix video session to infer **and plot** the resolution at 10-second time intervals.